### Yolo Mini Project
비눗방울 객체탐지

1차적으로 데이터를 수집하여 annotation을 1차적으로 진행했으며, 수집한 방식은 아래와 같다.
- 구글에서 약 50장 가량의 데이터 수집하고, 다이소에서 구매한 버블건을 활용하여 실제 사진을 찍기도 함
- LabelImg를 활용한 어노테이션 진행
- 72장의 train images, 21개의 validation images, 17개의 test images 수집 완료

사전에 훈련을 진행해보았을 때, 너무 작은 비눗방울의 경우 제대로된 탐지를 하지 못하는 문제점이 있었다.

즉시 어노테이션 파일을 점검한 결과로 어노테이션을 수정하고, 좀 더 타이트하게 어노테이션을 진행했다. 

<div align=center><img src="./Wrong Annotation.png"></div>

이후 학습을 진행했을 때 작은 물체까지 잘 잡아내는 모습을 보였으나, 이에 대한 정확도를 높이기 위해 추가적으로 데이터를 수집한다.

### 필요 라이브러리 import

In [1]:
import os
import cv2
import yaml
import torch
import shutil
from crawler.pix_crawling import PixCrawler  # 직접 만든 클래스
from imutils.video import WebcamVideoStream

### Image Url 수집
기존의 이미지와 픽사베이에서 수집한 이미지를 병합한다.

In [3]:
users_input = ["Bubble"]  # 여러개를 검색하려면 요소 추가할 것
keywords = users_input.split(", ")

for key in keywords:
    if key.startswith(" "):
        key.replace(key[0], "")
    else:
        pass
    
my_crawler = PixCrawler(keywords = keywords, num_of_data=200, path_datasets = "./Append", path_output_datasets = "./output")
my_crawler.dowload_datasets()

Bubble 데이터 다운로드를 시작합니다.


percentage: 100%|===================| 176/176 [00:16<00:00, 10.50it/s]

Bubble 176개 중 176개 이미지를 다운받았습니다. 
이미지 다운로드가 완료되었습니다. ./Append폴더를 확인해주세요. 


### 이미지 1차 전처리
이미 있는 데이터들이거나, 상관없는 이미지들의 경우 삭제한다. 

### 이미지 이동
176장 중 불필요한 데이터를 제거하고 총 34장의 데이터를 확보했다. 이전 결과와 똑같이 비교하기 위해 28장은 훈련셋 이미지로, 6장은 검증셋 이미지로 사용하며 테스트셋에는 별도로 더 추가하지 않는다.

In [4]:
image_dir = "./Append/Bubble"
image_list = os.listdir(image_dir)

for img in image_list[:28]:
    shutil.move(f"{image_dir}/{img}", f"./BubbleData/train/images/{img}")
    
for img2 in image_list[28:]:
    shutil.move(f"{image_dir}/{img2}", f"./BubbleData/validation/images/{img2}")

### 추가 데이터에 대한 라벨링 작업
추가적인 데이터에 대한 라벨링 작업을 진행한다.

In [5]:
%cd C:\Users\OWNER\Desktop\labeling\labelImg-master
!python labelimg.py

C:\Users\OWNER\Desktop\labeling\labelImg-master
[('bubble', [(150, 70), (582, 70), (582, 501), (150, 501)], None, None, False)]
Image:D:\Intel\YoloProject\BubbleData\train\images\1_Bubble.jpg -> Annotation:D:/Intel/YoloProject/BubbleData/train/labels\1_Bubble.txt
Image:D:\Intel\YoloProject\BubbleData\train\images\1_Bubble.jpg -> Annotation:D:/Intel/YoloProject/BubbleData/train/labels\1_Bubble.txt
[('bubble', [(112, 159), (164, 159), (164, 216), (112, 216)], None, None, False), ('bubble', [(84, 439), (141, 439), (141, 480), (84, 480)], None, None, False), ('bubble', [(198, 148), (236, 148), (236, 181), (198, 181)], None, None, False), ('bubble', [(169, 116), (220, 116), (220, 169), (169, 169)], None, None, False), ('bubble', [(149, 140), (183, 140), (183, 172), (149, 172)], None, None, False), ('bubble', [(149, 108), (174, 108), (174, 138), (149, 138)], None, None, False), ('bubble', [(129, 99), (175, 99), (175, 146), (129, 146)], None, None, False), ('bubble', [(246, 275), (284, 275), 

In [6]:
%cd D:\Intel\YoloProject

D:\Intel\YoloProject


### clone yolov5

In [7]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


### Yaml 파일 수정

In [8]:
with open('./yolov5/data/coco.yaml', "r") as f:
    data = yaml.safe_load(f)
    
data["path"] = r"D:\Intel\YoloProject\BubbleData"
data["train"] = "train"
data["test"] = "test"
data["val"] = "validation"

data["nc"] = 1
data["names"] = ["bubble"]

with open("./yolov5/data/coco.yaml", "w") as file:
    yaml.dump(data, file)

In [9]:
%pwd

'D:\\Intel\\YoloProject'

### 훈련
clone한 파일들 중 `train.py`를 이용해서 훈련을 진행한다. argparse로 작성되어 여러 옵션들을 함께 지정해주어야 한다.
- --batch : batch_size
- --epochs : 훈련 횟수
- --data : data정보를 담고있는 yaml 파일
- --weights : pretrained 된 가중치를 적용할 때 사용. 지정하지 않으면 랜덤한 가중치와 편향값들로 학습 진행. 해당 파일이 존재하지 않을 경우 다운로드를 받고 사용한다.
- --cfg : 모델을 담고있는 yaml파일. s ➡️ n ➡️ m ➡️ l ➡️ x 순으로 복잡도가 다르다. 

In [ ]:
!python ./yolov5/train.py --batch 2 --epochs 150 --data ./yolov5/data/coco.yaml  --weights ./yolov5/yolov5m.pt --cfg ./yolov5/models/yolov5m.yaml

### 검출
위에서 훈련한 나만의 모델을 기반으로 새로운 이미지들에 대한 객체 탐지 결과를 `detect.py`파일로 받아본다. 마찬가지로 argparse로 작성되어있다.
- --source : 탐지할 이미지 경로
- --weights : 나의 학습 모델의 가중치 경로
- --conf : 객체 탐지 신뢰도. 해당 신뢰도 이상의 객체들만 탐지한다. 기본값은 0.4이며 높을 수록 탐지되는 객체의 수도 적어진다.
- --name : 탐지된 파일들을 저장할 경로

In [ ]:
!python ./yolov5/detect.py --source "D:\Intel\YoloProject\BubbleData\test\images" --weights "./yolov5/runs/train/exp/weights/best.pt" --conf 0.65 --name "C:\Users\OWNER\Desktop\bubble_detection_test\results"

In [13]:
model = torch.hub.load('./yolov5', 'custom', path = "./yolov5/runs/train/exp/weights/best.pt", source = "local")
model.conf = 0.65

esp_ip = "http://192.168.137.213"
host = "{}:4747/video".format(esp_ip)      
cam = WebcamVideoStream(src=host).start()    

while True:
    frame = cam.read()
    results = model(frame)
    results.render()
    cv2.imshow('Modi Camera Module', frame)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

YOLOv5  v7.0-214-g8c30c58 Python-3.9.12 torch-1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


KeyboardInterrupt: 